In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

import timm  

In [2]:
DATA_PATH = '/kaggle/input/dog-breed-identification'
IMG_SIZE = 384  # Увеличенный размер для современных моделей
BATCH_SIZE = 32  # Уменьшен из-за увеличения размера изображения
EPOCHS = 15
LEARNING_RATE = 2e-4
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

Using device: cuda


In [3]:
# Подготовка
labels_df = pd.read_csv(f'{DATA_PATH}/labels.csv')
le = LabelEncoder()
labels_df['breed_encoded'] = le.fit_transform(labels_df['breed'])
num_classes = len(le.classes_)

class DogDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.iloc[idx]['id'] + '.jpg')
        image = Image.open(img_path).convert('RGB')
        label = self.df.iloc[idx]['breed_encoded']
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [4]:
# Аугментация данных
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
# Разделение данных
train_df, val_df = train_test_split(
    labels_df, 
    test_size=0.2, 
    stratify=labels_df['breed'],
    random_state=42
)

train_dataset = DogDataset(
    train_df, 
    img_dir=f'{DATA_PATH}/train', 
    transform=train_transform
)

val_dataset = DogDataset(
    val_df, 
    img_dir=f'{DATA_PATH}/train', 
    transform=val_transform
)

train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=2
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=2
)

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision

def show_images(loader, 
               class_names=None, 
               mean=[0.485, 0.456, 0.406], 
               std=[0.229, 0.224, 0.225],
               n_images=4,
               figsize=(10, 5)):
    images, labels = next(iter(loader))
    
    images = images[:n_images]
    labels = labels[:n_images]
    
    images = denormalize(images, mean, std)
    
    grid = torchvision.utils.make_grid(images, nrow=min(n_images, 4))
    
    npimg = grid.numpy()
    plt.figure(figsize=figsize)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    
    if class_names:
        title = ' | '.join([class_names[label] for label in labels.numpy()])
        plt.title(title, fontsize=14)
    
    plt.axis('off')
    plt.show()

def denormalize(tensor, mean, std):
    tensor = tensor.clone()
    
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
        
    return tensor

In [ ]:
dog_classes = list(labels_df['breed'].unique())

show_images(
    train_loader,
    class_names=dog_classes, 
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225],
    n_images=8,
    figsize=(15, 8)
)

In [8]:
def create_pretrained_model(num_classes):
    backbone = timm.create_model('convnext_small_in22k', 
                                pretrained=True, 
                                num_classes=0)
    
    classifier = nn.Sequential(
        nn.LayerNorm(backbone.num_features),
        nn.Linear(backbone.num_features, 1024),
        nn.GELU(),
        nn.Dropout(0.5),
        nn.Linear(1024, num_classes)
    )
    
    model = nn.Sequential(backbone, classifier)
    
    for param in model.parameters():
        param.requires_grad = False
    
    for param in classifier.parameters():
        param.requires_grad = True
    
    all_blocks = []
    for stage in backbone.stages:
        for block in stage.blocks:
            all_blocks.append(block)
    
    for block in all_blocks[-20:]:
        for param in block.parameters():
            param.requires_grad = True
    
    return model

# Инициализация модели
model = create_pretrained_model(num_classes).to(DEVICE)


all_blocks = []
for stage in model[0].stages:  
    for block in stage.blocks: 
        all_blocks.append(block)

# Размораживаем последние 20 блоков
for block in all_blocks[-20:]:
    for param in block.parameters():
        param.requires_grad = True

# Все параметры головы обучаемые
for param in model[1].parameters():
    param.requires_grad = True

optimizer = optim.AdamW([
    {'params': model[0].parameters(), 'lr': LEARNING_RATE / 10},
    {'params': model[1].parameters(), 'lr': LEARNING_RATE}
], weight_decay=0.05)

scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=EPOCHS * len(train_loader), 
    eta_min=1e-6
)

/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name convnext_small_in22k to current convnext_small.fb_in22k.
  model = create_fn(


model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

In [9]:
def test(model, loader):
    loss_log = []
    acc_log = []
    model.eval()

    for data, target in tqdm(loader, desc="Testing", leave=False):
        data, target = data.to(DEVICE), target.to(DEVICE)

        with torch.no_grad():
            output = model(data)
            loss = F.cross_entropy(output, target)

        pred = output.argmax(dim=1)
        acc = (pred == target).float().mean()

        loss_log.append(loss.item())
        acc_log.append(acc.item())

    return np.mean(loss_log), np.mean(acc_log)

def train_epoch(model, optimizer, train_loader):
    loss_log = []
    acc_log = []
    model.train()

    for data, target in tqdm(train_loader, desc="Training", leave=False):
        data, target = data.to(DEVICE), target.to(DEVICE)

        optimizer.zero_grad()

        output = model(data)
        loss = F.cross_entropy(output, target)

        loss.backward()
        optimizer.step()

        pred = output.argmax(dim=1)
        acc = (pred == target).float().mean()

        loss_log.append(loss.item())
        acc_log.append(acc.item())

    return loss_log, acc_log

def train(model, optimizer, n_epochs, train_loader, val_loader, scheduler=None):
    train_loss_log, train_acc_log, val_loss_log, val_acc_log = [], [], [], []
    best_val_loss = float('inf') 

    for epoch in range(n_epochs):
        train_loss, train_acc = train_epoch(model, optimizer, train_loader)
        
        val_loss, val_acc = test(model, val_loader)
        
        train_loss_log.extend(train_loss)
        train_acc_log.extend(train_acc)
        val_loss_log.append(val_loss)
        val_acc_log.append(val_acc)
        
        epoch_train_loss = np.mean(train_loss)
        epoch_train_acc = np.mean(train_acc)
        
        print(f"\nEpoch {epoch+1}/{n_epochs}")
        print(f"Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model_loss.pth')
            print(f"Saved best model with val_loss: {val_loss:.4f}")
        
        if scheduler:
            scheduler.step(val_loss)
    
    return train_loss_log, train_acc_log, val_loss_log, val_acc_log

In [10]:
# Запуск обучения
history = train(
    model, 
    optimizer, 
    EPOCHS, 
    train_loader, 
    val_loader, 
    scheduler=scheduler
)


Epoch 1/15
Train Loss: 2.3569 | Train Acc: 0.4981
Val Loss: 0.6533 | Val Acc: 0.8165


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Saved best model with val_loss: 0.6533



Epoch 2/15
Train Loss: 0.9662 | Train Acc: 0.7270
Val Loss: 0.5386 | Val Acc: 0.8362
Saved best model with val_loss: 0.5386



Epoch 3/15
Train Loss: 0.8308 | Train Acc: 0.7637
Val Loss: 0.4649 | Val Acc: 0.8518
Saved best model with val_loss: 0.4649


...


Epoch 11/15
Train Loss: 0.4868 | Train Acc: 0.8645
Val Loss: 0.4364 | Val Acc: 0.8689



Epoch 12/15
Train Loss: 0.4405 | Train Acc: 0.8767
Val Loss: 0.4543 | Val Acc: 0.8596



Epoch 13/15
Train Loss: 0.4512 | Train Acc: 0.8758
Val Loss: 0.4553 | Val Acc: 0.8625



Epoch 14/15
Train Loss: 0.4373 | Train Acc: 0.8760
Val Loss: 0.4705 | Val Acc: 0.8689



Epoch 15/15
Train Loss: 0.4055 | Train Acc: 0.8883
Val Loss: 0.4912 | Val Acc: 0.8581


In [11]:
# Создание предсказаний для submission
def create_submission(model, test_dir, transform, device, breed_names):
    test_files = os.listdir(test_dir)
    model.load_state_dict(torch.load('best_model_loss.pth'))
    model.eval()
    
    all_preds = []
    test_ids = []
    
    for file in tqdm(test_files, desc="Processing Test Images"):
        img_path = os.path.join(test_dir, file)
        image = Image.open(img_path).convert('RGB')
        image = transform(image).unsqueeze(0).to(device)
        
        with torch.no_grad():
            output = model(image)
            probas = F.softmax(output, dim=1).cpu().numpy()[0]
        
        all_preds.append(probas)
        test_ids.append(file.split('.')[0])
    
    submission = pd.DataFrame(
        np.vstack(all_preds),
        columns=breed_names
    )
    submission.insert(0, 'id', test_ids)
    return submission

# Генерация submission файла
test_dir = f'{DATA_PATH}/test'
breed_names = le.classes_
submission = create_submission(model, test_dir, val_transform, DEVICE, breed_names)
submission.to_csv('submission.csv', index=False)
print('Submission file created!')


Processing Test Images: 100%|██████████| 10357/10357 [05:09<00:00, 33.42it/s]


Submission file created!
